In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from sklearn.preprocessing import Normalizer
import cv2

In [13]:
norm_angles = angles / 360

# extract from file

In [70]:
img = cv2.imread('/home/me/Programmierung/AI_Test/find_arrow_symbol/detect_arrow/50_cm.jpg')
blurred = cv2.blur(img, (3,3))
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
retval, bw_img = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
cnts, _ = cv2.findContours(bw_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)  # arrow seems to be internal

point_list = []

for num, contour in enumerate(cnts):
    M = cv2.moments(contour)
    if (M["m00"] != 0):
        x_c = int(M["m10"] / M["m00"])
        y_c = int(M["m01"] / M["m00"])
    else:
        x_c = -1
        y_c = -1
    retval = cv2.arcLength(contour, True)
    points = cv2.approxPolyDP(contour, 0.04 * retval, True)
    print(len(points))
    if len(points) == 5:
        point_list.append(points.reshape((5,2)))
        
    #return (points, x_c, y_c)
print('#######')
print(len(point_list))


5
4
1
9
1
5
6
6
6
5
6
7
4
8
7
6
6
4
6
1
4
6
5
#######
4


In [50]:
for num in range(len(point_list)):
    if len(point_list[num]) == 5:
        img2 = np.zeros((480, 640,3))
        cv2.drawContours(img2, cnts, num, (0,0,255), thickness=10)
        #cv2.imshow('orig', img)
        cv2.imshow('test', img2)
        cv2.waitKey(0)
cv2.destroyAllWindows()

In [95]:
arr = np.array(point_list)
np.save('test', arr)


In [97]:
arr2 = np.load('test.npy')

In [98]:
arr.shape

(4, 5, 2)

In [99]:
arr2.shape

(4, 5, 2)

# extract from video for the first time

In [3]:
video_path = '/home/me/Videos/Kamera/2023-09-01-163208.webm'

In [6]:
def extract_good_values(frame):
    blurred = cv2.blur(frame, (3,3))
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    retval, bw_img = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    
    #import pdb
    #pdb.set_trace()
    
    cnts, _ = cv2.findContours(bw_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    point_list = []
    
    for num, contour in enumerate(cnts):
        #print(contour)
        M = cv2.moments(contour)
        if (M["m00"] != 0):
            x_c = int(M["m10"] / M["m00"])
            y_c = int(M["m01"] / M["m00"])
        else:
            x_c = -1
            y_c = -1
        retval = cv2.arcLength(contour, True)
        points = cv2.approxPolyDP(contour, 0.04 * retval, True)
        if len(points) == 5:
            point_list.append(points.reshape((5,2)))

        
    return point_list
        
        
        

In [11]:
found_shapes_raw = []
wrong_shapes_raw = []

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    cap.release()
    raise SystemExit()
    
frameRate = cap.get(5) #frame rate
abort = False

while cap.isOpened():
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if ret == True:
 
        # Display the resulting frame
        cv2.imshow('Frame',frame)
 
        key = cv2.waitKey(25) & 0xFF
    
        # Press Q on keyboard to  exit
        if key == ord('q') or abort:
            break
        elif frameId % np.floor(frameRate) == 0:
            try:
                values = extract_good_values(frame)
            except:
                print('an error occurred')
                raise SystemExit()
                
            for num, features in enumerate(values):
                con_frame = frame.copy()
                cv2.drawContours(con_frame, values, num, (0,0,255), thickness=10)
                cv2.imshow('Frame',con_frame)
                key = cv2.waitKey(0) & 0xFF
                if key == ord('n'):
                    wrong_shapes_raw.append(features)
                elif key == ord('y'):
                    found_shapes_raw.append(features)
                elif key == ord('q'):
                    abort = True
                    break
                    
                
    # Break the loop
    else: 
        break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

good_values = np.array(found_shapes_raw)
bad_values = np.array(wrong_shapes_raw)
np.save('good_values', good_values)
np.save('bad_values', bad_values)
print('done')


done


In [8]:
cv2.destroyAllWindows()
cap.release()

In [12]:
cap.isOpened()

False

In [13]:
loaded_good_values = np.load('good_values.npy')

In [14]:
loaded_good_values.shape

(121, 5, 2)

# minAreaRect, transform, scaling, ...

## get better shape data

In [16]:
shape_reference = np.load('shape_reference.npy')

In [22]:
M = cv2.moments(shape_reference)
x_c = int(M["m10"] / M["m00"])
y_c = int(M["m01"] / M["m00"])
retval = cv2.arcLength(shape_reference, True)
points = cv2.approxPolyDP(shape_reference, 0.04 * retval, True)

In [24]:
np.save('shape_reference_points', points)

In [10]:
points = np.load('shape_reference_points.npy')

In [11]:
reference_rect = cv2.minAreaRect(points)
# x,y,w,h = cv.boundingRect(cnt)
# center (x,y), (width, height), angle of rotation ( 90 degree is up), negative values possible
# center (x,y), (width, height), angle of rotation ( 90 degree is up), negative values possible
ref_box = cv2.boxPoints(reference_rect)
ref_box = np.intp(ref_box)

In [23]:
reference_rect

((291.92498779296875, 218.60198974609375),
 (135.26010131835938, 409.2398376464844),
 6.668918609619141)

In [24]:
black = np.zeros((480, 640, 4))
cv2.drawContours(black, points, -1, (255, 0, 0), thickness=10)
cv2.drawContours(black, [ref_box], -1, (0, 0, 255), thickness=5)
cv2.imshow('shape_reference', black)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
np.save('shape_reference_width_and_height', reference_rect[1])

In [27]:
ref_width, ref_height = np.load('shape_reference_width_and_height.npy')

In [28]:
ref_width

135.26010131835938

In [29]:
ref_height

409.2398376464844

## example

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from sklearn.preprocessing import Normalizer
import cv2

In [16]:
loaded_good_values = np.load('good_values.npy')

In [17]:
good_raw_features = loaded_good_values[0] 

In [18]:
good_raw_features

array([[266, 139],
       [322, 135],
       [386, 361],
       [339, 451],
       [269, 380]], dtype=int32)

In [36]:
min_rect = cv2.minAreaRect(good_raw_features)
# center (x,y), (width, height), angle of rotation ( 90 degree is up, clockwise), negative values possible
min_rect

((326.535888671875, 293.0493469238281),
 (316.1871337890625, 117.22742462158203),
 89.28681945800781)

In [32]:
box = cv2.boxPoints(min_rect)
box = np.intp(box)

In [21]:
test_rect = (*min_rect[:2], 45)
box = cv2.boxPoints(test_rect)
box = np.intp(box)

In [38]:
black = np.zeros((480, 640, 4))
cv2.drawContours(black, [good_raw_features], -1, (255, 0, 0), thickness=5)
cv2.drawContours(black,[box],0,(0,0,255),2)
cv2.imshow('example', black)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [46]:
def rot_point(min_rect):
    width_to_height = min_rect[1][0] / min_rect[0][0]
    
    box = cv2.boxPoints(test_rect)
    box = np.intp(box)
    
    lowest_point= np.arry((0, 1_000_000))
    second_lowest_point = np.arry((0, 2_000_000))
    for i in range(len(box)):
        if box[i][1] < lowest_point[1]:
            lowest_point = box[i]
        elif box[i][1] < second_lowest_point[1]:
            second_lowest_point = box[i]
        
    if widht_to_height > 0:
        if 0 <= min_rect[2] < 90:
            # lowest point, rot further
            offset_point = lowest_point
            needed_rot = 90 - min_rect[2]
        else:
            # second lowest point, rot back
            offset_point = second_lowest_point
            needed_rot = 90 - min_rect[2]
            
    else:
        if 0 < min_rect[2] <= 90:
            # second lowest point, rot back
            offset_point = second_lowest_point
            needed_rot = -min_rect[2]
            
        else:
            # lowest point, rot further
            offset_point = lowest_point
            needed_rot = 180 - min_rect[2]
            
    return offset_point, needed_rot

In [47]:
# remove offset

In [48]:
# rotate

In [41]:
box

array([[265, 135],
       [383, 134],
       [387, 450],
       [269, 451]])

# svm opencv (svm.save for saving)

In [29]:
import cv2 as cv
import numpy as np
# Set up training data
labels = np.array([1, -1, -1, -1])
trainingData = np.matrix([[501, 10], [255, 10], [501, 255], [10, 501]], dtype=np.float32)
# Train the SVM
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(trainingData, cv.ml.ROW_SAMPLE, labels)
# Data for visual representation
width = 512
height = 512
image = np.zeros((height, width, 3), dtype=np.uint8)
# Show the decision regions given by the SVM
green = (0,255,0)
blue = (255,0,0)
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        sampleMat = np.matrix([[j,i]], dtype=np.float32)
        response = svm.predict(sampleMat)[1]
        response = response[0][0]
        
        if response == 1:
            image[i,j] = green
        elif response == -1:
            image[i,j] = blue
        else:
            import pdb
            pdb.set_trace()
        
#import pdb
#pdb.set_trace()
        
# Show the training data
thickness = -1
cv.circle(image, (501, 10), 5, ( 0, 0, 0), thickness)
cv.circle(image, (255, 10), 5, (255, 255, 255), thickness)
cv.circle(image, (501, 255), 5, (255, 255, 255), thickness)
cv.circle(image, ( 10, 501), 5, (255, 255, 255), thickness)
# Show support vectors
thickness = 2
sv = svm.getUncompressedSupportVectors()
for i in range(sv.shape[0]):
    cv.circle(image, (int(sv[i,0]), int(sv[i,1])), 6, (128, 128, 128), thickness)
    

#cv.imwrite('result.png', image) # save the image
cv.imshow('SVM Simple Example', image) # show it to the user
cv.waitKey()
cv.destroyAllWindows()